In [1]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../src/')
from evaluate import get_results
from omegaconf import OmegaConf
from main import load_data, make_model
import numpy as np
import os
import glob
import demap
from tqdm import tqdm
# from evaluation import compute_encoding_metrics, get_dataset_contents, get_noiseless_name, get_ambient_name, get_data_config, eval_results, compute_recon_metric
from evaluation import compute_all_metrics, get_noiseless_name, get_ambient_name
from transformations import NonTransform

In [2]:
# Initialize wandb (replace 'your_entity' and 'your_project' with your specific details)
wandb.login()
api = wandb.Api()

# Specify your entity, project, and sweep ID
entity = "xingzhis"
project = "dmae"
sweep_id = 'wgysuau8'

# Fetch the sweep
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

run_ids = [run.id for run in sweep.runs]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


In [3]:
run = sweep.runs[0]

In [4]:
cfg = OmegaConf.create(run.config)
folder_path = "../src/wandb/"
try:
    folder_list = glob.glob(f"{folder_path}*{run.id}*")
    ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
    ckpt_path = ckpt_files[0]
except:
    print(f"No checkpoint found for run {run.id}")
cfg = OmegaConf.create(run.config)
data_root = '../synthetic_data2/'
data_path = os.path.join(data_root, cfg.data.name + cfg.data.filetype)
noiseless_path = os.path.join(data_root, get_noiseless_name(cfg.data.name) + cfg.data.filetype)
ambient_path = os.path.join(data_root, get_ambient_name(cfg.data.name) + '.npy')

In [5]:
pp = NonTransform()
emb_dim = cfg.model.emb_dim
dist_std = 1.
input_dim = 100
model = make_model(cfg, input_dim, emb_dim, pp, dist_std, from_checkpoint=True, checkpoint_path=ckpt_path)

In [6]:
res_dict = compute_all_metrics(model, data_path, noiseless_path, ambient_path)

In [7]:
res_dict

{'seedmethod': 'groups,46',
 'bcv': '0.2',
 'dropout': '0.7',
 'demap': 0.7140251186997651,
 'accuracy': 0.7504084617128057,
 'recon score': 0.6243550985072934}

In [8]:
pd.DataFrame([res_dict, res_dict])

,seedmethod,bcv,dropout,demap,accuracy,recon score
0,"groups,46",0.2,0.7,0.714025,0.750408,0.624355
1,"groups,46",0.2,0.7,0.714025,0.750408,0.624355
